# End to end multi class dog breed classification 
this notebook builds the multiclass classification using the tensorflow's latest version which is 2.4.1 
and tensorflow hub 

## problem 
Identify the breeds of the dogs given an image of the dog 
## data 
data is imported from the kaggle competition 

## evalutation 
    evaluation is the file with the prediction probabilities of each dog breed of the test images 

## features
some info about data 
* we are dealing with the images (120) breeds of the dogs 
* we are using the deep learning/ trnasfer learning ( this is the multiclass classification ) 
* there are around the 10000 + images in the training set and 10000 in the test set 
* training set images are having the labels but the testing dataset are not having any labels 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('../input/dog-breed-identification/labels.csv')

In [ ]:
len(data)

In [ ]:
# importing the data and getting the workspace ready 
import tensorflow as tf 
print('tf version ',tf.__version__)

In [ ]:
import tensorflow_hub as hub
print('tf_hub version' , hub.__version__)

In [ ]:
print('GPU available ' if tf.config.list_physical_devices('GPU') else print('its not avail'))

In [ ]:

import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

In [ ]:
# if we do get the runtime disconnected model restart with runnning all the cells 
#  getting the data ready (converting the images into tensor's ) 
data.head()

In [ ]:
data.describe()

In [ ]:
data['breed'].value_counts().plot.bar(figsize=(20,10))

In [ ]:
data['breed'].value_counts().median()

In [ ]:
# viewing the images inside the notebook
from IPython.display import Image
Image('../input/dog-breed-identification/train/001513dfcb2ffafc82cccf4d8bbaba97.jpg')

### getting the images and their labels 
getting the list of the image file pathnames 


In [ ]:
data.head()

In [ ]:
# creating the pathnames from image id's
filenames = ['../input/dog-breed-identification/train/'+fname+'.jpg' for fname in data['id'] ]
filenames

In [ ]:
Image(filenames[-1:][0])

In [ ]:
import os 
if len(os.listdir('../input/dog-breed-identification/train/')) == len(filenames):
    print('filenames match the actual amount ')
else : 
    print('filenames do not match the actual amt ')

In [ ]:
Image(filenames[9000])

In [ ]:
data['breed'][9000]

In [ ]:
# let's prepare the lables 
labels = np.array(data.breed)
labels

In [ ]:
len(labels)

In [ ]:
if len(labels) == len(filenames):
    print('all the data can be mapped ')

In [ ]:
unique_breeds = np.unique(labels)
len(unique_breeds)

In [ ]:
print(labels[0])
labels[0] == unique_breeds 

In [ ]:
# turning the labels into a boolean array 
boolean_labels =  [label == unique_breeds for label in labels ]

In [ ]:
len(boolean_labels)

In [ ]:
# turnign the boolean array into the integers 
print(labels[0])
print(np.where(unique_breeds == labels[0])) # index where the label occurs 
print(boolean_labels[0].argmax()) # index where label occurs in boolean array 
print(boolean_labels[0].astype(int)) # there will be 1 where the sample occurs

In [ ]:
# creating our own validation set 
x = filenames
y = boolean_labels

In [ ]:
# setting the number of images for experimenting 
NUM_IMAGES = 1000  #@param {type:'slider',min:1000,max:10000,step:1000}

In [ ]:
# splitting the data into train and validation 
from sklearn.model_selection import train_test_split
# splitting the data into 2 different sets 
x_train,x_val, y_train,y_val = train_test_split(x[:1000],y[:1000],test_size = 0.2,random_state = 42)

In [ ]:
len(x_train), len(x_val), len(y_train), len(y_val)

In [ ]:
x_train[:5],y_train[:2]

In [ ]:
# preprocessing images turning the images into the tensor's
# using the filepath as input 

# conver the image into the numpy array with importing the image 
from matplotlib.pyplot import imread 
image = imread(filenames[42])
image.shape

In [ ]:
tf.constant(image)

In [ ]:
# writing the function to turn the image into tensors tensors are execute faster with gpu's 
IMG_SIZE = 224 
def process_image(image_path,img_size = IMG_SIZE):
    image = tf.io.read_file(image_path)
    # turning the jpeg image into numberical tensor in 3 color channels 
    image = tf.image.decode_jpeg(image,channels=3)
    # converting the color channel values from 0-255 to 0-1 values 
    image = tf.image.convert_image_dtype(image,tf.float32)
    # resizing the image into the specified 224 
    image = tf.image.resize(image,size = [img_size,img_size])
    return image 

In [ ]:
# turnign the data into the batches of 32 tensorflow executes batches more efficiently cause all the 1000 image might not fit into the memory 
# create a simple function to get the label 
def get_image_label(image_path,label):
    image  = process_image(image_path)
    return image,label


In [ ]:
# turning the images into the batches 
BATCH_SIZE = 32 
# creating the function to turn data into batches 
def create_data_batches(x,y=None,batch_size = BATCH_SIZE,valid_data = False,test_data = False):
    #if the data is the data is the test data set then we don't have the labels 
    if test_data : 
        print('creating the test data batches')
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only file paths no labels 
        data_batch = data.map(process_image).batch(batch_size)
        return data_batch 
    # if the data is the validation dataset we don't need to shuffle the validation data set 
    elif valid_data:
        print('creating the validation dataset batches ')
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))# file paths and labels 
        data_batch = data.map(get_image_label).batch(batch_size)
        return data_batch
    else:
        print('creating the training databatches')
        # turning the file path and label's into tensors 
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x),tf.constant(y)))
        # shuffling pathnames and labels before mapping images to the processor function is faster than shuffling images
        data = data.shuffle(buffer_size=len(x))
        data = data.map(get_image_label)
        #turning the training data into batches 
        data_batch = data.batch(batch_size)
        return data_batch
        

In [ ]:
# creating the training data and the validation data batches 
train_data = create_data_batches(x_train,y_train)
val_data = create_data_batches(x_val,y_val,valid_data=True)

In [ ]:
 train_data.element_spec,val_data.element_spec 

In [ ]:
# visualizing the data batches 
#  data is now in the batches but it's hard to understand let's visualize them 

import matplotlib.pyplot as plt 
#  creating the function of viewing the images in the data batch 
def show_25_images(images,labels):
    plt.figure(figsize = (10,10))
    # loop through the 25 images 
    for i in range(25):
        # creating the subplots 5 rows and 5 cols 
        ax = plt.subplot( 5,  5 , i+1)
        plt.imshow(images[i]) #showing the ith image 
        plt.title(unique_breeds[labels[i].argmax()])
        plt.axis('off') # turning the gridlines off 
        

In [ ]:
train_images,train_labels = next(train_data.as_numpy_iterator())
len(train_images), len(train_labels)

In [ ]:
# let's visualize the data in the training batch 
show_25_images(train_images,train_labels)

In [ ]:
# let's visualize the validation data 
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images,val_labels)

In [ ]:
# building the model 
# before we build a model there are few things which we need to define 
# the input shape in the form of the tensors to our model 
#  the output shape image, labels in the form of the tensors 
# URL of the model which we want to use 


In [ ]:
# setting up the input shape to the model 
INPUT_SHAPE = [None,IMG_SIZE,IMG_SIZE,3 ] # batch , height width and the color channels 

OUTPUT_SHAPE = len(unique_breeds)

# SETTING  the model url with the tensorflow hub 
MODEL_URL = 'https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/classification/4' # we get the model from tensorflow hub 

In [ ]:
# creating the ML model function let's put them together into keras deep learning model 
#  getting the functional into line by line 
# the func takes the input shape, output shape and the model we choosen as parameters and defines the layers in keras in sequential model 
# compiles the model (how it should be evaluated/ improved )
# builds the model tells the input shape which it's getting 
#  returns the model all the steps can be referred to keras overview in tensorflow website 

In [ ]:

def create_model(input_shape=INPUT_SHAPE,output_shape = OUTPUT_SHAPE,model_url = MODEL_URL):
    print('building the model with ' + model_url)
    # setting up the layers 
    model = tf.keras.Sequential([
        hub.KerasLayer(model_url),# layer 1 input layer 
        tf.keras.layers.Dense(units=output_shape,activation='softmax') # layer 2 output layer 
        
    ])
    #compile the model 
    model.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics = ['accuracy']
    )
    model.build(input_shape)
    return model 

In [ ]:
model = create_model()
model.summary()

In [ ]:
outputs = np.ones(shape=(1,1,200))
outputs

In [ ]:
#  creating the callback's 
# callbacks are the helper function which in runtime helps the model to save it's progress , check it's progress, stop training early if the model stops 
# we are creating 2 callback's one for the tensorboard for checking the progress and one for the early stopping which will prevent the model for running too long 
# tensorboard callback 
# Load Tensorboard notebook extension 
%load_ext tensorboard 

# to set up the tensorboard callback we will need to do 3 things 
1. loading the tensorboard notebook extension 
2. creating the tensorboard callbacks which will save the logs to the directory 
    passign the model to the fit function
3. visualizing the model's training log with tensorboard magic function 

In [ ]:
import datetime
# creating the function to build a tensorboard call back  
def create_tensorboard_callback():
    logdir = os.path.join("./logs",datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
    return tf.keras.callbacks.TensorBoard(logdir)

In [ ]:
# training the model 
#  the first model is gonna train on the thousand images to making sure everything is working fine 
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',patience=3)

In [ ]:
NO_EPOCHS = 100 
# checking to make sure we are working on the gpu 
print('GPU Avail YAYYYYYYYYYYYY' if tf.config.list_physical_devices('GPU') else print("NA"))

# creating the function to train the model 
setting up the tensorboard using create_tensorboard_callback 
calling the fit function for training data , val data, no of epochs  adn the callbacks which will help the func

In [ ]:
# Build the function to train and return the trained model 
def train_model():
    model = create_model()
    # creating the new tensorboard sessoin everytime we create the model 
    tensorboard = create_tensorboard_callback()
    # fitting the model to the data passing the callbacks we created 
    model.fit(x=train_data,
             epochs = NO_EPOCHS,
             validation_data = val_data,
             validation_freq=1,
             callbacks=[tensorboard,early_stopping])
    #returning the fitted model 
    return model 

In [ ]:
 #fitting the model to the data 
model =train_model()

In [ ]:
# checking the tensorboard logs 
#  %tensorboard used to access the data 
%tensorboard --logdir ./logs

In [ ]:
# making and evaluating the predictions with the trained model 
val_data

In [ ]:
predictions = model.predict(val_data,verbose=1)
predictions

In [ ]:
predictions.shape

In [ ]:
len(y_val)

In [ ]:
len(unique_breeds)

In [ ]:
print(predictions[0])

In [ ]:
def predict_probas(index):
    print(predictions[index])
    print(f'max prediction is : {np.max(predictions[0])}')
    print('sum', np.sum(predictions[0]))
    print('max Index : ', np.argmax(predictions[index]))
    print('predicted label ;',unique_breeds[np.argmax(predictions[index])])
    print('actual label ' )

In [ ]:
predict_probas(0)

In [ ]:
for i in range(10):
    predict_probas(i)

In [ ]:
# prediction probabilities are the confidence levels 
def get_predict_label(predict ):
    return unique_breeds[np.argmax(predict)]


In [ ]:
get_predict_label(0)

In [ ]:
val_data

In [ ]:
# we will have to unbatch the data to get our actual validation predictions 
images = []
labels = []

for image, label in val_data.unbatch().as_numpy_iterator():
    images.append(image)
    labels.append(label)

In [ ]:
from matplotlib.pyplot import imshow
imshow(images[0]),get_predict_label(labels[0]),get_predict_label(predictions[0])

In [ ]:
def unbatch_data(data):
    images = []
    labels = []
    
    
    
    for image,label in data.unbatch().as_numpy_iterator():
        images.append(image)
        labels.append(labels)
    return images, labels
val_images, val_labels = unbatch_data(val_data)

In [ ]:
# i = 1 
# plt.subplot(1,1,i)
# plt.imshow(images[i])
# plt.title('predicted:' + str(get_predict_label(predictions[i])) + 'actual :' + str(get_predict_label(val_labels[i])) )

In [ ]:
def plot_pred(predictions_probabilities , labels, images,n=1):
    pred_prob,true_label,image = predictions_probabilities[1],labels[1],images[1]
    # get the pred lables 
    pred_label = get_predict_label(pred_prob)
#     plt.imshow(image)
#     plt.xticks([])
#     plt.yticks([])
    
    print(str(pred_label)+str(np.max(pred_prob)*100)+str(unique_breeds[np.argmax(true_label)]))
    

In [ ]:
plot_pred(predictions_probabilities=predictions,labels=val_labels,images=images)

In [ ]:
% timeit 
print('hello world ')